# Transformers 模型量化技术：GPTQ

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

2022年，Frantar等人发表了论文 [GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)。

这篇论文详细介绍了一种训练后量化算法，适用于所有通用的预训练 Transformer模型，同时只有微小的性能下降。

GPTQ算法需要通过对量化模型进行推理来校准模型的量化权重。详细的量化算法在[原始论文](https://arxiv.org/pdf/2210.17323.pdf)中有描述。

基于[`auto-gptq`](https://github.com/PanQiWei/AutoGPTQ.git) 开源实现库，transformers 支持使用GPTQ算法量化的模型。

## 使用 GPTQ 量化模型

为了使用 `auto-gptq` 库量化一个模型，我们需要向量化器传递一个数据集。

通常有两种方式构造数据集：
- 量化器支持的默认数据集（包括`['wikitext2','c4','c4-new','ptb','ptb-new']`）
- 一个字符串列表（这些字符串将被用作数据集）

### 使用 GPTQ 算法支持的默认数据集来量化

在下面的示例中，让我们尝试使用`"wikitext2"`数据集将模型量化为4位精度。支持的精度有`[2, 4, 6, 8]`。

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

#### 逐层量化

关于 `CUDA extension not installed` 的说明: https://github.com/AutoGPTQ/AutoGPTQ/issues/249

In [ ]:
!pip install optimum
!pip install bitsandbytes
!pip install optimum

In [ ]:
import sys
import os
import torch
import transformers
import bitsandbytes
import accelerate

print("--- System and Library Versions ---")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"BitsAndBytes version: {bitsandbytes.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

print("\n--- CUDA Availability ---")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"PyTorch CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")
    # Check NVIDIA driver version (requires nvidia-smi to be in PATH)
    try:
        import subprocess
        driver_output = subprocess.check_output("nvidia-smi --query-gpu=driver_version --format=csv,noheader", shell=True).decode().strip()
        print(f"NVIDIA Driver Version: {driver_output}")
    except Exception:
        print("Could not retrieve NVIDIA driver version (nvidia-smi not found or error).")
else:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!! WARNING: CUDA IS NOT AVAILABLE. Quantization WILL NOT WORK. !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

print("\n--- BitsAndBytes CUDA Binding Check ---")
try:
    from bitsandbytes.cuda_setup.main import get_compute_capability
    compute_cap = get_compute_capability()
    print(f"BitsAndBytes CUDA Compute Capability: {compute_cap}")
    print("BitsAndBytes seems to have found its CUDA binding successfully.")
except Exception as e:
    print(f"ERROR: Could not get compute capability from bitsandbytes: {e}")
    print("This indicates a severe issue with bitsandbytes' CUDA binding or installation.")
    print("!!! Quantization WILL NOT WORK until this is resolved. !!!")

print("\n--- Environment Variables (relevant ones) ---")
print(f"LD_LIBRARY_PATH (Linux/macOS): {os.environ.get('LD_LIBRARY_PATH', 'Not set')}")
print(f"PATH (partial, first 5 entries): {os.environ.get('PATH', 'Not set').split(os.pathsep)[:5]}...")

In [ ]:
!pip install --upgrade optimum

In [ ]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

### 实测 GPTQ 量化模型：GPU显存占用峰值超过10GB

```shell
Wed Feb  7 22:03:42 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   66C    P0              69W /  70W |   9719MiB / 15360MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A      2334      C   /root/miniconda3/bin/python               10714MiB |
+---------------------------------------------------------------------------------------+
```

### 检查量化模型正确性

通过检查`线性层的属性`来确保模型已正确量化，它们应该包含`qweight`和`qzeros`属性，这些属性应该是`torch.int32`数据类型。

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

In [ ]:
# 保存模型权重
quant_model.save_pretrained("models/opt-2.7b-gptq")

#### 使用 GPU 加载模型并生成文本

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still here.
Thank you! I'm glad to be here too.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

下面演示通过传递自定义数据集来量化一个模型。

通过字符串列表来自定义一个数据集，建议样本数不少于128（样本数太少会影响模型性能）

In [ ]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

相比使用默认数据集，未经精心准备的自定义数据集会明显降低模型性能

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

## [Optional]Homework: 使用 GPTQ 算法量化 OPT-6.7B 模型
